In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px # beautiful graphics
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Task
#### Predict future (2015-11) sales
#### Metric is RMSE
#### Submition file `ID, item_cnt_month`

## Подготовка данных

In [ ]:
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv', index_col = 'shop_id')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv', index_col = 'item_category_id')
categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv', index_col = 'item_category_id')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv', index_col = 'shop_id')

Объединение товаров и категорий  
Объединение продаж и магазинов

In [ ]:
items = items.join(categories, on='item_category_id')
items.reset_index(inplace=True)
sales = sales.join(shops, on='shop_id')
sales.reset_index(inplace=True)

Полное объединение импортированных сущностей

In [ ]:
sales.set_index('item_id', inplace=True)
items.set_index('item_id', inplace=True)
sales = sales.join(items, on='item_id')
sales.reset_index(inplace=True)

In [ ]:
sales.item_price = sales.item_price.apply(abs)
sales.item_cnt_day= sales.item_cnt_day.apply(abs)

In [ ]:
sales['date'] = pd.to_datetime(sales['date'])

Неправильная разметка данных создала коллизии. В датасете имеются даты старше ноября 15 года

In [ ]:
sales.loc[sales[sales.date >= '11.01.2015'].index, 'date'] = sales[sales.date >= '11.01.2015']['date']\
    .agg(lambda x: x.dt.strftime('%Y-%d-%m'))

In [ ]:
sales.info()

In [ ]:
sales.describe()

In [ ]:
sales.head()

## EDA

#### Посмотрим ближе на продажи в день

In [ ]:
sales.item_cnt_day.describe()

Проверим, являются ли максимальные значения выбросами

In [ ]:
sales[sales.item_cnt_day >= 1e3]

In [ ]:
pd.DataFrame(np.stack(np.unique(sales.item_cnt_day, return_counts=True)).T)

In [ ]:
px.bar(pd.DataFrame(np.stack(np.unique(sales.item_cnt_day, return_counts=True)).T, columns=['count','freq']), 
       log_y=True, log_x=True, x='count', y='freq', template='plotly_dark')

Значение выглядят вполне адекватно, можно двигаться дальше

### Цена товара

In [ ]:
sales.item_price.describe()

Проверка максимума

In [ ]:
sales[sales.item_price > 1e5] ## Явный выброс

Стандартная лицензия Radmin 3 стоит ~1500 ₽ на 25.04.2022  
Заменим значение на его же, но поделенного на 0.05% квантиль распределения цен

In [ ]:
sales.at[1163158, 'item_price'] = sales.at[1163158, 'item_price'] / sales.item_price.quantile(.05)

In [ ]:
px.histogram(sales.item_price.unique(), template='plotly_dark')

In [ ]:
sales[sales.item_price > sales.item_price.quantile(.99)].sort_values('item_price').tail(10)

In [ ]:
sales[sales.item_price < 1].sort_values('item_price').head(10)

**Итог**: цены и продажи выглядят вполне адекватно и не имеют явных выбросов

### Даты - основа временных рядов. Посмотрим на распределение записей по датам


In [ ]:
group = sales.groupby('date')

In [ ]:
dates = pd.DataFrame(group.date.count()).rename(columns={'date':'count'})

In [ ]:
px.histogram(dates.reset_index(), x='date', y='count', template='plotly_dark', nbins=1034//30)

#### Имеется тенденция к сокращению продаж или просто перестали вносить в базу данных новые записи?

In [ ]:
counts = pd.DataFrame(group.item_cnt_day.sum()).rename(columns={'item_cnt_day':'sum'})

In [ ]:
print(f'Процент корреляции между суммарными продажами и количеством записей: {counts.corrwith(dates["count"])[0]:.2%}')

#### Падение числа количества записей почти эквивалентно падению суммарных продаж
> P.s.
Среднее не используется, так как на гистограмме распределения продаж видно, что подавляющее большинство это 1 продажа.

In [ ]:
px.histogram(counts.reset_index(), x='date', y='sum', template='plotly_dark', nbins=1034//30)

#### С чем может быть связана эта тенденция?  
Можно предположить, что с ценой, которая способна меняться во времени.

Но какую меру использовать для оценки цены товаров во времени?  
Count ничем не будет отличаться от количества записей, а сумма будет привязана к количеству продаж.  
Логично использовать среднюю цену товара в конкретный день.

In [ ]:
items = pd.DataFrame(group.item_price.mean()).rename(columns={'item_price':'mean'})

In [ ]:
px.histogram(items.reset_index(), x='date', y='mean', template='plotly_dark', nbins=1034//30)

In [ ]:
print(f'Процент корреляции между суммарными продажами и количеством записей: {dates.corrwith(items["mean"])[0]:.2%}')

#### Теперь можно видеть что зависимость между падением числа записей отчасти обусловлена ростом цен.

#### Промежуточный итог
* Цены и продажи имеют корреляцию
* Явных выбросов нет, но необходимо искать латентные (для каждой пары `{товар, магазин}`)
* Можно видеть невооруженным глазом, что существует зависимость между активностью пользователя и месяцем

### С чем может быть связано падение цен?
На ум сразу приходит кризис [Russian financial crisis](https://en.wikipedia.org/wiki/Russian_financial_crisis_(2014%E2%80%932016)), при котором отношение USD/RUB выросло с 33 до 66 почти за полтора года

Используем данные о курсе доллара, чтобы выявить положительные корреляции
> P.s.
Используется датасет на основе данных [центробанка](https://www.cbr.ru/currency_base/dynamics/)

In [ ]:
usd = pd.read_csv('../input/usd-for-1c/USD.csv', index_col=['date'], parse_dates=['date'])

In [ ]:
new = pd.concat((usd, items, counts, dates), join='inner', axis=1)

In [ ]:
new

In [ ]:
new.corr()

#### По корреляционной матрице выше можно сделать несколько предположений:
1. Цены на товары начали расти вместе с ростом доллара.
2. Количество продаж снизилось, когда доллар стал расти.
3. Люди стали менее активно совершать покупки из-за роста доллара.

#### Обогащение данных
Представленный датасет можно обогатить при помощи данных из открытых источников. К примеру, мы можем использовать погодные данные и территориальные.  
Можно предположить, что, при плохих погодных условиях, продажи могут снижаться. Так же, на интуитивном уровне кажется, что расстояние до центра города может быть взаимосвязанно с уровнем продаж.

In [ ]:
shops.shop_name.head()

In [ ]:
mos = pd.read_csv('../input/russia-weather/Moscow.csv', parse_dates=True, index_col='date')

In [ ]:
mos_shop = sales[sales.shop_id.isin(
    [shop_id for shop_id, shop_name in shops.iterrows() if 'Москва' in shop_name[0]]
)].groupby('date').item_cnt_day.sum()

In [ ]:
mos = mos.merge(mos_shop, how='right', on='date').fillna(method='bfill')

In [ ]:
mos.corr()

In [ ]:
mos_shop = shops.loc[[shop_id for shop_id, shop_name in shops.iterrows() if 'Москва' in shop_name[0]],:]

In [ ]:
mos_shop['dist'] = [6.2, 6.1, None, 9, 9, 2.5, 15, 20, 21, 16, 19, 7.3, 7] ## km

In [ ]:
mos_shop['count'] = sales[sales.shop_id.isin(
    [shop_id for shop_id, shop_name in shops.iterrows() if 'Москва' in shop_name[0]]
)].groupby('shop_id').item_cnt_day.count()

In [ ]:
mos_shop.corr()

In [ ]:
del mos_shop, mos, shops, new, items, counts, dates, group, categories

#### Матрицы корреляций выше не дали ожидаемых результатов, однако, отсутствие линейной корреляции - не признак того, что данные не взаимосвязаны.

## Аггрегация по месяцу

> Для того чтобы не повторять дальнейшие вычисления каждый раз при запуске среды, измененный датасет сохранен в виде parquet.gzip файла.  
Далее приведен код, который был запущен для получения аггрегированных по месяцам данных о продажах.  

Импортируем удаленные из памяти датасеты магазинов и товаров и посмотрим на будущий индекс, размер которого будет ```34*22170*60```.
```python
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv', 
                    index_col = 'item_category_id').reset_index()
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv', 
                    index_col = 'shop_id').reset_index()
### Посмотрим на будущий индекс
pd.MultiIndex.from_product(
    [pd.date_range('2013-01', '2015-11', freq='M'), shops.shop_id.unique(), items.item_id.unique()]
)
```
Создадим датафрейм с полным индексом, к которому мы впоследствии и будем присоединять существующие данные.
```python
### Создадим датафрейм с полным индексом
index = pd.DataFrame(data=None,
    index=pd.MultiIndex.from_product(
        (pd.date_range('2013-01', '2015-11', freq='M'), shops.shop_id.unique(), items.item_id.unique()),
        names=('date', 'shop_id', 'item_id')
    )
)

### Зададим индекс для данных о продажах
### И используем его, чтобы преобразовать данные к необходимому формату

sales = sales.set_index(['date', 'shop_id', 'item_id'], drop=False)

monthly = sales.groupby([pd.Grouper(level=0, freq='M'), pd.Grouper(level=1), pd.Grouper(level=2)]).agg(
    {
        'item_price':'mean', 'item_cnt_day':'sum', 'item_category_id':'last',
        'shop_id':'last', 'item_id':'last',
    }
)
### Проверим, совпадают ли индексы
monthly.index.levels[0] == index.index.levels[0]
```
Присоединим методом ```left join``` данные к датафрейму. Данный вид соединения используется для того, чтобы "растянуть" существующий датафрейм до нужного нам размера. Появившиеся пропущенные значения мы будем заполнять позже.
```python
### Используем left join, чтобы заполнить null значениями недостающие данные
new = index.join(monthly, on=['date','shop_id','item_id'], how='left')

### 96% данных - отсутствуют в изначальном датасете

### Пометим записи, которые отсутствовали в изначальном датасете
new['null'] = new.isna().sum(1) != 0

### Установим отсутствующие продажи на ноль
new.item_cnt_day.fillna(0, inplace=True)

### Удалим соответствующие колонки, чтобы присоединить их уже заполненными
new = new.drop(['shop_id', 'item_id', 'item_category_id'], axis=1).reset_index()

items.set_index('item_id', inplace=True)
new.set_index('item_id', inplace=True)

### Присоединяем и сбрасываем индекс
new = new.join(items.item_category_id, how='left', on='item_id')

new.reset_index(inplace=True)
```
Далее мы создаем pivot таблицу на основе нового датасета, чтобы постепенно заполнять пропущенные значения в ценах. В качестве индексов и колонок выступали различные комбинации date, shop_id, item_id. Это сделано для того, чтобы заполнить все пропущенные значения цен как можно качественее.
1. Сначала мы заполнили цены для пар {товар, магазин}, однако таких пар не хватило, чтобы покрыть весь индекс. ```['date'] ['shop_id','item_id']```
2. Затем мы использовали заполнение цен для товаров в конкретный временной промежуток ```['shop_id'] ['date','item_id']```
3. После этого мы заполняли значения без учета времени и магазина ```['date', 'shop_id'] ['item_id']```

```python
pivot = new.pivot(index=index, columns=columns, values='item_price')

### Сначала заполняем предыдущими значениями, а затем будущими, чтобы закрыть все nan
pivot = pivot.fillna(method='bfill').fillna(method='ffill')

### Присоединяем датафрейм с правильным индексом
new = new.join(
    pd.melt(pivot, ignore_index=False).reset_index().set_index(['date','shop_id','item_id']), 
    on=['date','shop_id','item_id'], how='left'
)

### Присваиваем новые значения для цен
### И удаляем лишнюю колонку
new['item_price'] = new['value']

new.drop('value', axis=1, inplace=True)
```
После проделанных манипуляций, строк с пропущенными значениями осталось около 750000, это 2% от всего объема. В связи с чем было принято решение заполнить пропуски внутрегрупповыми средними.
```python
### Оставшиеся пропущенные значения цен мы заполнили средней ценой в группе
### Исходя из предположения, что цены внутри категории не сильно отличаются
group = new.groupby('item_category_id').item_price.mean()

new['item_price'] = new.item_price.fillna(
    new[new.item_price.isna()].item_category_id.apply(lambda id: group[id])
)
```

In [ ]:
msales = pd.read_parquet('../input/m1cparquet/msales.gzip')

In [ ]:
msales

### Временной ряд
Ниже приведен классический временной ряд из цен и продаж пар {магазин, товар}

In [ ]:
ts = msales.pivot(index=['shop_id', 'item_id'], columns=['date'], values=['item_cnt_month', 'item_price'])

In [ ]:
ts.head()

#### Корреляции
Измерив корреляцию между продажами, мы не можем сделать однозначного вывода, так как мы не знаем в полной мере, обусловлено ли это малым количеством продаж, отсутствием **линейной** взаимосвязи или ещё какими-нибудь факторами.

In [ ]:
ts.corr()

Уберем колонку с ценой

In [ ]:
ts = ts.drop('item_price', axis=1)
ts.columns = ts.columns.droplevel(0)
ts

Вместо mean можно использовать любую функцию для предсказания

In [ ]:
ts.apply('mean', axis=1)

### Погружение
Не долго думая стоит сразу погрузиться в данные и осмотреться - не самый быстрый и интересный способ, однако один из самых эффективных.

In [ ]:
series = msales[(msales.shop_id == 7) & (msales.item_id == 5821)]

In [ ]:
series.corr()['item_cnt_month']['item_price']

In [ ]:
px.line(series, x='date', y='item_cnt_month', template='plotly_dark')

In [ ]:
series = msales[(msales.shop_id == 50) & (msales.item_id == 7893)]

In [ ]:
series.corr()['item_cnt_month']['item_price']

In [ ]:
px.line(series, x='date', y='item_cnt_month', template='plotly_dark')

На графиках выше видно, с чем могу быть связаны полученные оценки корреляции. Различные временные ряды имеют разную тенденцию.  
В пользу этой гипотезы может свидетельствовать и само значения корреляции ```0±малое значение```. Такой результат может возникать в том числе, если ряды будут компенсировать корреляции друг друга (как на графиках выше).

## Гипотезы

### Let's take a look...
Сколько пар {магазин, товар} не продавались никогда

In [ ]:
zeros_id = ts[ts.sum(axis=1) == 0].index

In [ ]:
print(f"Среди всех пар {{магазин, товар}} около {zeros_id.shape[0] / ts.shape[0]:.0%} не имели продаж")

Проведем тест, чтобы оценить прирост качества предсказания, если мы будем и дальше предсказывать 0 для товаров без продаж, а оставшеися предсказания заполним случайным числом.

In [ ]:
y = ts.loc[:, '2015-10-31'].copy(True)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
filled = y.copy(True)
### Изменения составляют 5-10%
filled[::] = np.random.random()
print(mean_squared_error(y, filled, squared=False))
filled[zeros_id] = 0
print(mean_squared_error(y, filled, squared=False))

In [ ]:
nonsaled_id = ts[ts.iloc[:, -4:-1].sum(axis=1) == 0].index

In [ ]:
print(f"Среди всех пар {{магазин, товар}} около {nonsaled_id.shape[0] / ts.shape[0]:.0%} не имели продаж в последние 3 месяца")

In [ ]:
filled = y.copy(True)
### Изменения в районе 10-20%
filled[::] = np.random.random()
print(mean_squared_error(y, filled, squared=False))
filled[nonsaled_id] = 0
print(mean_squared_error(y, filled, squared=False))

### Промежуточный итог
Как можно видеть по результатам выше, меньшинство товаров очень сильно влияют на метрику, так что, грубая оценка или предсказания константой явно не подойдут.

Далее импортируем расширенный тест Дики-Фуллера для оценки стационарности рядов.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
non_zero = ts[~ts.index.isin(zeros_id)] ### Находим все пары, у которых НЕ нулевые продажи

In [ ]:
### Функция для apply
### Проверяем на стационарность
### Если test[0] - результат
### Больше test[4] - 1% критического значения
### То ряд не стационарен
def non_stationarity(x):
    test = adfuller(x)
    return test[0] > test[4]['1%']

Поскольку время выполнения зависит от метода, который не умеет взаимодействовать с несколькими рядами, из сторонней библиотеки statsmodels, мы используем параллелльные вычисления, чтобы сократить ожидание до 15 минут.
```python
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
### 15 минут
%time non_zero['non_stationary'] = non_zero.parallel_apply(non_stationarity, axis=1)
### Выведем количество стационарных временных рядов в датасете
((non_zero.non_stationary == False).sum() + zeros_id.shape[0]) / ts.shape[0]
```

#### Подавляющее большинство временных рядов - стационарны

In [ ]:
print(f"Среди всех пар {{магазин, товар}} около {msales.stationary.sum() / msales.shape[0]:.0%} являются стационарными")